In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.fft import fft, ifft
from scipy.signal import periodogram, welch
import ordpy 
import statsmodels.api as sm
from scipy.signal import butter,filtfilt
from matplotlib.pyplot import figure
from os import listdir
from os.path import isfile, join
from statsmodels.tsa.stattools import adfuller, pacf
from tqdm.notebook import tqdm
import math
from scipy.stats import entropy
from scipy import interpolate
import ptitprince as pt
import seaborn as sns
import pingouin
import nolds
import EntropyHub as EH

# General Functions

In [2]:
def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier
def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

def flatten(seq):
    l = []
    for elt in seq:
        t = type(elt)
        if t is tuple or t is list:
            for elt2 in flatten(elt):
                l.append(elt2)
        else:
            l.append(elt)
    return l

# Functions for analysis

In [3]:
#mean, variance
def mean(rating):
    mean_list = []
    for i in range(len(rating)):
        if len(rating[i]) > 1:
            mean_list.append(np.mean(rating[i]))
        else:
            mean_list.append(np.NaN)
    return mean_list


def variance(rating,time):
    variance_list = []
    for i in range(len(rating)):
        if len(rating[i]) > 1:
            variance = np.var(rating[i])
            variance_list.append(variance)
        else:
            variance_list.append(np.NaN)
    return variance_list

def var_all(rating):
    '''Calculate the variance of the concacenated time series'''
    if len(rating) == 1:
        rating_collected = rating[0]
    elif len(rating) == 2:
        rating_collected = np.hstack([rating[0], rating[1]])
    elif len(rating) == 3:
        rating_collected = np.hstack([rating[0], rating[1], rating[2]])
    elif len(rating) == 4:
        rating_collected = np.hstack([rating[0], rating[1], rating[2], rating[3]])
    elif len(rating) == 5:
        rating_collected = np.hstack([rating[0], rating[1], rating[2], rating[3], rating[4]])
    else: 
        rating_collected = np.nan
    return np.var(rating_collected)

def cv(rating):
    cv_list = []
    for i in range(len(rating)):
        cv_list.append(np.std(rating[i])/np.mean(rating[i]))
    return cv_list
def cv_all(rating):
    '''Calculate the CV of the whole concacenated time series'''
    if len(rating) == 1:
        rating_collected = rating[0]
    elif len(rating) == 2:
        rating_collected = np.hstack([rating[0], rating[1]])
    elif len(rating) == 3:
        rating_collected = np.hstack([rating[0], rating[1], rating[2]])
    elif len(rating) == 4:
        rating_collected = np.hstack([rating[0], rating[1], rating[2], rating[3]])
    elif len(rating) == 5:
        rating_collected = np.hstack([rating[0], rating[1], rating[2], rating[3], rating[4]])
    else: 
        rating_collected = np.nan
    return np.std(rating_collected)/np.mean(rating_collected)

#power-spectrum - welch method
def power_spectrum(rating,time):
    psd_freq_list = []
    psd_list = []
    for i in range(len(rating)):
        if len(rating[i]) > 1:
            N = len(rating[i])
            T = time[i][-1]
            f, S = welch(rating[i], 20)
            psd_freq_list.append(f)
            psd_list.append(S)
        else:
            psd_freq_list.append([np.NaN])
            psd_list.append([np.NaN])
    return psd_freq_list, psd_list


#permutation entropy - complexity
def permutation_entropy(rating):
    entropy_list = []
    stat_complexity_list = []
    for i in range(len(rating)):
        if len(rating[i]) > 1:
            entropy, stat_complexity = ordpy.complexity_entropy(rating[i])
            entropy_list.append(entropy)
            stat_complexity_list.append(stat_complexity)
        else:
            entropy_list.append(np.NaN)
            stat_complexity_list.append(np.NaN)
    return entropy_list, stat_complexity_list

#sample entropy
def sample_entropy_func_default(rating):
    entropy_list = []
    for i in range(len(rating)):
        if len(rating[i]) > 1:
            entropy, A, B = EH.SampEn(rating[i])
            entropy_list.append(entropy)
        else:
            entropy_list.append(np.NaN)
    return entropy_list

#autocorrelation
def autocorrelation(rating, time):
    acorr_list = []
    time_lag_list = []
    for i in range(len(rating)):
        if len(rating[i]) > 1:
            acorr = sm.tsa.acf(rating[i], nlags = int(len(rating[i])))
            acorr_list.append(acorr)
        else:
            acorr_list.append([np.NaN, np.NaN, np.NaN])
    for i in range(len(time)):
        if len(time[i]) > 1:
            time_lag = np.array(time[i]) - np.array(time[i][0])
            time_lag_list.append(time_lag)
        else:
            time_lag_list.append([np.NaN, np.NaN, np.NaN])
    return acorr_list, time_lag_list

#check stationarity
def adfuller_test(rating):
    adfuller_list = []
    stationary_list = []
    for i in range(len(rating)):
        if len(rating[i]) > 1:
            result = adfuller(rating[i])
            adfuller_list.append(result)
            if result[1] <= 0.05:
                stationary_list.append(1)
            else:
                stationary_list.append(0)
        else:
            adfuller_list.append(np.nan)
            stationary_list.append(np.nan)
    labels = ['ADF Test Statistic','p-value','#Lags Used','#Observation Used']
    return adfuller_list, labels, stationary_list
    
# Hurst Exponent with DFA method
def hurst_dfa(rating):
    '''Hurst exponent calculated wwith DFA method. Fractal Dimension is calculated as D = 2-H where H is the Hurst exponent'''
    poly_order = 1
    rating = np.random.randn(10000)#rating_all[0][0]
    window_sizes = [len(rating)//2,len(rating)//4,len(rating)//8,len(rating)//16,len(rating)//32]
    F_n_all = []
    for w in window_sizes:
        X_t = np.cumsum(np.array(rating) - np.mean(rating))
        segments = np.array_split(X_t, len(X_t)//w)
        poly_coeffs_all = []
        Y_t = []
        for i in range(len(segments)):
            poly_coeff = np.polyfit(np.arange(w), segments[i][:w], poly_order)
            poly_coeffs_all.append(poly_coeff) #slope then intercept
            Y_t_seg = poly_coeff[0]*np.arange(w)+poly_coeff[1]
            for j in range(len(Y_t_seg)):
                Y_t.append(Y_t_seg[j])
        F_n = np.sqrt(np.sum((X_t[:len(Y_t)] - np.array(Y_t))**2)/len(X_t))
        F_n_all.append(F_n)
    #plt.scatter(np.log(window_sizes),np.log(F_n_all))
    return np.polyfit(np.log(window_sizes),np.log(F_n_all),1)

In [4]:
data = pd.read_pickle('../Data/data_file/D1_puregooddata_split_31032023.pkl')
rating_all = []
for i in range(0,len(data)):
    rating_all.append(data['rating'].iloc[i])
timing_all = []
for i in range(0,len(data)):
    timing_all.append(data['time'].iloc[i])
participant_list = data['PID'].unique()
rating_downsample_list_all = rating_all
time_downsample_list_all = timing_all

# Calculations
## Mean, Variance, CV

In [5]:
mean_list_all = []
mean_square_list_all = []
rms_list_all = []
variance_list_all = []
std_list_all = []

for j in range(len(rating_downsample_list_all)):
    mean_list = mean(rating_downsample_list_all[j])
    variance_list= variance(rating_downsample_list_all[j], time_downsample_list_all[j])
    mean_list_all.append(mean_list)
    variance_list_all.append(variance_list)
mean_overall = []
for i in range(len(mean_list_all)):
    mean_overall.append(np.mean(mean_list_all[i]))
var_combined_list_all = []
for j in range(len(rating_downsample_list_all)):
    var_combined_list = var_all(rating_downsample_list_all[j])
    var_combined_list_all.append(var_combined_list)

cv_list_all = []

for j in range(len(rating_downsample_list_all)):
        cv_list = cv(rating_downsample_list_all[j])
        cv_list_all.append(cv_list)
        
cv_combined_list = []
for j in range(len(rating_downsample_list_all)):
    cv_list = cv_all(rating_downsample_list_all[j])
    cv_combined_list.append(cv_list)       

    


## PSD

In [ ]:

psd_freq_list_all = []
psd_list_all=[]


for j in range(len(rating_downsample_list_all)):
    psd_freq_list, psd_list = power_spectrum(rating_downsample_list_all[j], time_downsample_list_all[j])
    psd_freq_list_all.append(psd_freq_list)
    psd_list_all.append(psd_list)

## Entropy
### Permutation Entropy

In [ ]:
entropy_list_all = []
stat_complexity_list_all = []
for i in range(len(rating_downsample_list_all)):
    #plt.figure()
    entropy_list, stat_complexity_list = permutation_entropy(rating_downsample_list_all[i])
    entropy_list_all.append(entropy_list)
    stat_complexity_list_all.append(stat_complexity_list)
entropy_overall = []
for i in range(len(entropy_list_all)):
    entropy_overall.append(np.mean(entropy_list_all[i]))

### Sample Entropy

In [ ]:
sample_entropy_list_all = []
for i in tqdm(range(len(rating_downsample_list_all))):
    sample_entropy_list = sample_entropy_func_default(rating_downsample_list_all[i])
    sample_entropy_list_all.append(sample_entropy_list)
sample_entropy_overall = []
for i in range(len(sample_entropy_list_all)):
    sample_entropy_overall.append(np.mean(sample_entropy_list_all[i]))

## Check Stationarity

In [ ]:
adfuller_list_all = []
stationary_list_all = []
for j in range(len(rating_downsample_list_all)):
    adfuller_list, labels, stationary_list = adfuller_test(rating_downsample_list_all[j])
    adfuller_list_all.append(adfuller_list)
    stationary_list_all.append(stationary_list)



## Autocorrelation

In [ ]:
acorr_list_all = []
time_lag_list_all = []
for j in range(len(rating_downsample_list_all)):
        
    #plt.figure()
    acorr_list, time_lag = autocorrelation(rating_downsample_list_all[j], time_downsample_list_all[j])
    acorr_list_all.append(acorr_list)
    time_lag_list_all.append(time_lag)

In [ ]:
char_time_scale_index = []
char_time_scale = []
for i in range(len(acorr_list_all)):
    char_time_scale_index_part = []
    char_time_scale_part = []
    for j in range(len(acorr_list_all[i])):
        time_index = np.argmax(acorr_list_all[i][j] < 0.1)
        char_time_scale_index_part.append(time_index)
        char_time_scale_part.append(time_lag_list_all[i][j][time_index])

    char_time_scale_index.append(char_time_scale_index_part)
    char_time_scale.append(char_time_scale_part)
char_time_scale_mean = []
for i in range(len(char_time_scale)):
    char_time_scale_mean.append(np.mean(char_time_scale[i]))

## Fractal Dimension

In [ ]:
hurst_dfa_all = []
fractal_dfa_all = []
for i in range(len(rating_downsample_list_all)):
    hurst_dfa_list = []
    fractal_list = []
    for j in range(len(rating_downsample_list_all[i])):
        hurst_dllm = hurst_dfa(rating_downsample_list_all[i][j])
        hurst_dfa_list.append(hurst_dllm[0])
        fractal_list.append(2-hurst_dllm[0])
    hurst_dfa_all.append(hurst_dfa_list)
    fractal_dfa_all.append(fractal_list)


fractal_dfa_overall = []
for i in range(len(fractal_dfa_all)):
    fractal_dfa_overall.append(np.nanmean(fractal_dfa_all[i]))

# ARIMA model

In [ ]:
from pmdarima.arima import auto_arima
def calcsmape(actual, forecast):
    return 1/len(actual) * np.sum(2 * np.abs(forecast-actual) / (np.abs(actual) + np.abs(forecast)))

length_data = []
for i in range(len(rating_downsample_list_all)):
    for j in range(len(rating_downsample_list_all[i])):
        length_data.append(len(rating_downsample_list_all[i][j]))


TEST_SIZE = 100
smape_all = []
model_all = []
for i in tqdm(range(len(rating_downsample_list_all))):
    smape_participant = []
    model_participant = []
    for j in range(len(rating_downsample_list_all[i])):
        data_trial = pd.Series(rating_downsample_list_all[i][j])
        train, test = data_trial[:-TEST_SIZE], data_trial[-TEST_SIZE:]

        x_train, x_test = np.array(range(train.shape[0])), np.array(range(train.shape[0], data_trial.shape[0]))
        train.shape, x_train.shape, test.shape, x_test.shape
        model = auto_arima(train, start_p=1, start_q=1,
                            test='adf',
                            max_p=5, max_q=5,
                            m=1,             
                            d=1,          
                            seasonal=False,   
                            start_P=0, 
                            D=None, 
                            trace=True,
                            error_action='ignore',  
                            suppress_warnings=True, 
                            stepwise=True)
        model_participant.append(model)
        prediction, confint = model.predict(n_periods=TEST_SIZE, return_conf_int=True)
        cf= pd.DataFrame(confint)
        prediction_series = pd.Series(prediction, index = test.index)
        fig1, ax1 = plt.subplots(1, 1, figsize=(15, 5))
        ax1.plot(data_trial, label = 'original')
        ax1.plot(prediction_series, label = 'forecasting')
        ax1.fill_between(prediction_series.index,
                        cf[0],
                        cf[1],color='grey',alpha=.3)
        ax1.set_ylim(0,10)
        ax1.set_title(f'participant id {participant_list[i]}, day 1, trial {j+1}')
        ax1.legend()
        smape=calcsmape(test,prediction)
        smape_participant.append(smape)
    smape_all.append(smape_participant)
    model_all.append(model_participant)
smape_mean_all = []
for i in range(len(smape_all)):
    smape_mean_all.append(np.mean(smape_all[i]))

# Correlation Testing

In [9]:
#list all the data files and its path
questionnaire = []
onlyfiles = [f for f in listdir('../Data/questionnaire') if isfile(join('../Data/questionnaire', f))]
for i in range(len(onlyfiles)):
    path = '../Data/questionnaire/' + onlyfiles[i]
    questionnaire.append(path)


df_questionnaire_pre = pd.read_csv(questionnaire[0])
df_questionnaire_s1 = pd.read_csv(questionnaire[1])
df_questionnaire_s2 = pd.read_csv(questionnaire[2])
index_pre = []
for id in participant_list:
    index_pre.append(np.where(df_questionnaire_pre.PROLIFIC_PID == id)[0][0])
index_s1 = []
for id in participant_list:
    index_s1.append(np.where(df_questionnaire_s1.PROLIFIC_PID == id)[0])
index_s2 = []
for id in participant_list:
    index_s2.append(np.where(df_questionnaire_s2.PROLIFIC_PID == id)[0])
df_questionnaire_index_pre = df_questionnaire_pre.iloc[index_pre]
msk_scores = df_questionnaire_index_pre['Score'].values
msk_last_q_corr = np.load('msk_last_q.npy')
index_s1_normal = []
for i in range(len(index_s1)):
    if len(index_s1[i]) == 1:
        index_s1_normal.append(index_s1[i][0])

In [ ]:
df_s1_normal = df_questionnaire_s1.iloc[index_s1_normal]
participant_list_s1_questionnaire = df_s1_normal.PROLIFIC_PID.values
s1_questionnaire_index = []
for i in range(len(participant_list)):
    if participant_list[i] in participant_list_s1_questionnaire:
        s1_questionnaire_index.append(i)
mean_s1 = np.array(mean_overall)[s1_questionnaire_index]
var_s1 = np.array(var_combined_list_all)[s1_questionnaire_index]
entropy_s1 = np.array(entropy_overall)[s1_questionnaire_index]
cv_s1 = np.array(cv_combined_list)[s1_questionnaire_index]
acf_s1 = np.array(char_time_scale_mean)[s1_questionnaire_index]
smape_s1 = np.array(smape_mean_all)[s1_questionnaire_index]
sample_entropy_s1 = np.array(sample_entropy_overall)[s1_questionnaire_index]
index_s2_normal = []
for i in range(len(index_s2)):
    if len(index_s2[i]) == 1:
        index_s2_normal.append(index_s2[i][0])
df_s2_normal = df_questionnaire_s2.iloc[index_s2_normal]
participant_list_s2_questionnaire = df_s2_normal.PROLIFIC_PID.values#.to_list()
s2_questionnaire_index = []
for i in range(len(participant_list)):
    if participant_list[i] in participant_list_s2_questionnaire:
        s2_questionnaire_index.append(i)
mean_s2 = np.array(mean_overall)[s2_questionnaire_index]
var_s2 = np.array(var_combined_list_all)[s2_questionnaire_index]
entropy_s2 = np.array(entropy_overall)[s2_questionnaire_index]
cv_s2 = np.array(cv_combined_list)[s2_questionnaire_index]
acf_s2 = np.array(char_time_scale_mean)[s2_questionnaire_index]
smape_s2 = np.array(smape_mean_all)[s2_questionnaire_index]
sample_entropy_s2 = np.array(sample_entropy_overall)[s2_questionnaire_index]



## code to find correlation coefficient and BF10 - just change the name of the questionnaire

## pre-screening (msk_bf, msk_lastq_bf)

In [ ]:
print('mean')
mean_mskscores = zip(mean_overall, msk_scores)
x = []; y=[]
for point in list(mean_mskscores):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
x_bestfit = np.arange(0,11,0.1)
y_bestfit = a*x_bestfit+b
corr_mean_mskscores = np.corrcoef(x,y)
print(corr_mean_mskscores)
msk_bf = []
bf_mean_msk = pingouin.bayesfactor_pearson(corr_mean_mskscores[0][1], len(x))
msk_bf.append(bf_mean_msk)
print(bf_mean_msk)

print('---------')
print('variance')
var_mskscores = zip(var_combined_list_all, msk_scores)
x = []; y=[]
for point in list(var_mskscores):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_var_mskscores = np.corrcoef(x,y)
print(corr_var_mskscores)
bf_var_msk = pingouin.bayesfactor_pearson(corr_var_mskscores[0][1], len(x))
msk_bf.append(bf_var_msk)
print(bf_var_msk)


print('---------')
print('CV')
fano_mskscores = zip(cv_combined_list, msk_scores)
x = []; y=[]
for point in list(fano_mskscores):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_fano_mskscores = np.corrcoef(x,y)
print(corr_fano_mskscores)
bf_fano_msk = pingouin.bayesfactor_pearson(corr_fano_mskscores[0][1], len(x))
msk_bf.append(bf_fano_msk)
print(bf_fano_msk)

print('---------')
print('PE')
pe_mskscores = zip(entropy_overall, msk_scores)
x = []; y=[]
for point in list(pe_mskscores):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_pe_mskscores = np.corrcoef(x,y)
print(corr_pe_mskscores)
bf_entropy_msk = pingouin.bayesfactor_pearson(corr_pe_mskscores[0][1], len(x))
msk_bf.append(bf_entropy_msk)
print(bf_entropy_msk)

print('---------')
print('SampEn')
sampen_mskscores = zip(sample_entropy_overall, msk_scores)
x = []; y=[]
for point in list(sampen_mskscores):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_sampen_mskscores = np.corrcoef(x,y)
print(corr_sampen_mskscores)
bf_sampen_msk = pingouin.bayesfactor_pearson(corr_sampen_mskscores[0][1], len(x))
msk_bf.append(bf_sampen_msk)
print(bf_sampen_msk)

print('---------')
print('acf')
acf_mskscores = zip(char_time_scale_mean, msk_scores)
x = []; y=[]
for point in list(acf_mskscores):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_acf_mskscores = np.corrcoef(x,y)
print(corr_acf_mskscores)
bf_acf_msk = pingouin.bayesfactor_pearson(corr_acf_mskscores[0][1], len(x))
msk_bf.append(bf_acf_msk)
print(bf_acf_msk)

print('---------')
print('smape')
smape_mskscores = zip(smape_mean_all, msk_scores)
x = []; y=[]
for point in list(smape_mskscores):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_smape_mskscores = np.corrcoef(x,y)
print(corr_smape_mskscores)
bf_smape_msk = pingouin.bayesfactor_pearson(corr_smape_mskscores[0][1], len(x))
msk_bf.append(bf_smape_msk)
print(bf_smape_msk)






In [ ]:


print('mean')
mean_mskscores_lastq = zip(mean_overall, msk_last_q_corr)
x = []; y=[]
for point in list(mean_mskscores_lastq):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
x_bestfit = np.arange(0,11,0.1)
y_bestfit = a*x_bestfit+b
corr_mean_mskscores_lastq = np.corrcoef(x,y)
print(corr_mean_mskscores_lastq)
msk_lastq_bf = []
bf_mean_msk_lastq = pingouin.bayesfactor_pearson(corr_mean_mskscores_lastq[0][1], len(x))
msk_lastq_bf.append(bf_mean_msk_lastq)
print(bf_mean_msk_lastq)

print('---------')
print('variance')
var_mskscores_lastq = zip(var_combined_list_all, msk_last_q_corr)
x = []; y=[]
for point in list(var_mskscores_lastq):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_var_mskscores_lastq = np.corrcoef(x,y)
print(corr_var_mskscores_lastq)
bf_var_msk_lastq = pingouin.bayesfactor_pearson(corr_var_mskscores_lastq[0][1], len(x))
msk_lastq_bf.append(bf_var_msk_lastq)
print(bf_var_msk_lastq)


print('---------')
print('CV')
fano_mskscores_lastq = zip(cv_combined_list, msk_last_q_corr)
x = []; y=[]
for point in list(fano_mskscores_lastq):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_fano_mskscores_lastq = np.corrcoef(x,y)
print(corr_fano_mskscores_lastq)
bf_fano_msk_lastq = pingouin.bayesfactor_pearson(corr_fano_mskscores_lastq[0][1], len(x))
msk_lastq_bf.append(bf_fano_msk_lastq)
print(bf_fano_msk_lastq)

print('---------')
print('PE')
pe_mskscores_lastq = zip(entropy_overall, msk_last_q_corr)
x = []; y=[]
for point in list(pe_mskscores_lastq):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_pe_mskscores_lastq = np.corrcoef(x,y)
print(corr_pe_mskscores_lastq)
bf_entropy_msk_lastq = pingouin.bayesfactor_pearson(corr_pe_mskscores_lastq[0][1], len(x))
msk_lastq_bf.append(bf_entropy_msk_lastq)
print(bf_entropy_msk_lastq)

print('---------')
print('SampEn')
sampen_mskscores_lastq = zip(sample_entropy_overall, msk_last_q_corr)
x = []; y=[]
for point in list(sampen_mskscores_lastq):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_sampen_mskscores_lastq = np.corrcoef(x,y)
print(corr_sampen_mskscores_lastq)
bf_sampen_msk_lastq = pingouin.bayesfactor_pearson(corr_sampen_mskscores[0][1], len(x))
msk_lastq_bf.append(bf_sampen_msk_lastq)
print(bf_sampen_msk_lastq)

print('---------')
print('acf')
acf_mskscores_lastq = zip(char_time_scale_mean, msk_last_q_corr)
x = []; y=[]
for point in list(acf_mskscores_lastq):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_acf_mskscores_lastq = np.corrcoef(x,y)
print(corr_acf_mskscores_lastq)
bf_acf_msk_lastq = pingouin.bayesfactor_pearson(corr_acf_mskscores_lastq[0][1], len(x))
msk_lastq_bf.append(bf_acf_msk_lastq)
print(bf_acf_msk_lastq)

print('---------')
print('smape')
smape_mskscores_lastq = zip(smape_mean_all, msk_last_q_corr)
x = []; y=[]
for point in list(smape_mskscores_lastq):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_smape_mskscores_lastq = np.corrcoef(x,y)
print(corr_smape_mskscores_lastq)
bf_smape_msk_lastq = pingouin.bayesfactor_pearson(corr_smape_mskscores_lastq[0][1], len(x))
msk_lastq_bf.append(bf_smape_msk_lastq)
print(bf_smape_msk_lastq)






## section 1 (bpisev_bf, bpiint_bf, start_bf, startsub_bf)

In [ ]:
print('mean')
bpi_mean_severity = df_s1_normal['BPI Mean Severity Score'].values
mean_bpisev = zip(mean_s1, bpi_mean_severity)
x = []; y=[]
for point in list(mean_bpisev):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_mean_bpisev = np.corrcoef(x,y)
print(corr_mean_bpisev)
bpisev_bf = []
bf_mean_bpisev = pingouin.bayesfactor_pearson(corr_mean_bpisev[0][1], len(x))
bpisev_bf.append(bf_mean_bpisev)
print(bf_mean_bpisev)
print('------')
print('variance')
var_bpisev = zip(var_s1, bpi_mean_severity)
x = []; y=[]
for point in list(var_bpisev):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_var_bpisev = np.corrcoef(x,y)
print(corr_var_bpisev)
bf_var_bpisev = pingouin.bayesfactor_pearson(corr_var_bpisev[0][1], len(x))
bpisev_bf.append(bf_var_bpisev)
print(bf_var_bpisev)

print('------')
print('CV')
fano_bpisev = zip(cv_s1, bpi_mean_severity)
x = []; y=[]
for point in list(fano_bpisev):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_fano_bpisev = np.corrcoef(x,y)
print(corr_fano_bpisev)
bf_fano_bpisev = pingouin.bayesfactor_pearson(corr_fano_bpisev[0][1], len(x))
bpisev_bf.append(bf_fano_bpisev)
print(bf_fano_bpisev)

print('------')
print('PE')
entropy_bpisev = zip(entropy_s1, bpi_mean_severity)
x = []; y=[]
for point in list(entropy_bpisev):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_entropy_bpisev = np.corrcoef(x,y)
print(corr_entropy_bpisev)
bf_entropy_bpisev = pingouin.bayesfactor_pearson(corr_entropy_bpisev[0][1], len(x))
bpisev_bf.append(bf_entropy_bpisev)
print(bf_entropy_bpisev)

print('------')
print('SampEn')
sample_entropy_bpisev = zip(sample_entropy_s1, bpi_mean_severity)
x = []; y=[]
for point in list(sample_entropy_bpisev):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_sample_entropy_bpisev = np.corrcoef(x,y)
print(corr_sample_entropy_bpisev)
bf_sample_entropy_bpisev = pingouin.bayesfactor_pearson(corr_sample_entropy_bpisev[0][1], len(x))
bpisev_bf.append(bf_sample_entropy_bpisev)
print(bf_sample_entropy_bpisev)

print('------')
print('ACF')
acf_bpisev = zip(acf_s1, bpi_mean_severity)
x = []; y=[]
for point in list(acf_bpisev):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_acf_bpisev = np.corrcoef(x,y)
print(corr_acf_bpisev)
bf_acf_bpisev = pingouin.bayesfactor_pearson(corr_acf_bpisev[0][1], len(x))
bpisev_bf.append(bf_acf_bpisev)
print(bf_acf_bpisev)

print('------')
print('smape')
smape_bpisev = zip(smape_s1, bpi_mean_severity)
x = []; y=[]
for point in list(smape_bpisev):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_smape_bpisev = np.corrcoef(x,y)
print(corr_smape_bpisev)
bf_smape_bpisev = pingouin.bayesfactor_pearson(corr_smape_bpisev[0][1], len(x))
bpisev_bf.append(bf_smape_bpisev)
print(bf_smape_bpisev)


In [ ]:
bpi_mean_severity = df_s1_normal['BPI Mean Severity Score'].values
fano_bpisev = zip(cv_s1, bpi_mean_severity)
x = []; y=[]
for point in list(fano_bpisev):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_fano_bpisev = np.corrcoef(x,y)
print(corr_fano_bpisev)
bf_fano_bpisev = pingouin.bayesfactor_pearson(corr_fano_bpisev[0][1], len(x))
print(bf_fano_bpisev)
a, b = np.polyfit(x, y, 1)
x_bestfit = np.arange(0,2,0.1)
y_bestfit = a*x_bestfit+b

plt.rcParams["figure.figsize"] = (8,4)
fig, ax = plt.subplots()
ax.scatter(x,y)
ax.set_xlabel('CV')
ax.set_ylabel('score')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_title('BPI mean severity score against CV')

In [ ]:
print('mean')
bpi_mean_interference = df_s1_normal['BPI Mean Pain Interference Score'].values
mean_bpiint = zip(mean_s1, bpi_mean_interference)
x = []; y=[]
for point in list(mean_bpiint):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_mean_bpiint = np.corrcoef(x,y)
print(corr_mean_bpiint)
bpiint_bf = []
bf_mean_bpiint = pingouin.bayesfactor_pearson(corr_mean_bpiint[0][1], len(x))
bpiint_bf.append(bf_mean_bpiint)
print(bf_mean_bpiint)

print('------')
print('variance')
var_bpiint = zip(var_s1, bpi_mean_interference)
x = []; y=[]
for point in list(var_bpiint):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_var_bpiint = np.corrcoef(x,y)
print(corr_var_bpiint)
bf_var_bpiint = pingouin.bayesfactor_pearson(corr_var_bpiint[0][1], len(x))
bpiint_bf.append(bf_var_bpiint)
print(bf_var_bpiint)


print('------')
print('CV')
fano_bpiint = zip(cv_s1, bpi_mean_interference)
x = []; y=[]
for point in list(fano_bpiint):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_fano_bpiint = np.corrcoef(x,y)
print(corr_fano_bpiint)
bf_fano_bpiint = pingouin.bayesfactor_pearson(corr_fano_bpiint[0][1], len(x))
bpiint_bf.append(bf_fano_bpiint)
print(bf_fano_bpiint)

print('------')
print('PE')
entropy_bpiint = zip(entropy_s1, bpi_mean_interference)
x = []; y=[]
for point in list(entropy_bpiint):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_entropy_bpiint = np.corrcoef(x,y)
print(corr_entropy_bpiint)
bf_entropy_bpiint = pingouin.bayesfactor_pearson(corr_entropy_bpiint[0][1], len(x))
bpiint_bf.append(bf_entropy_bpiint)
print(bf_entropy_bpiint)

print('------')
print('SampEn')
sample_entropy_bpiint = zip(sample_entropy_s1, bpi_mean_interference)
x = []; y=[]
for point in list(sample_entropy_bpiint):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_sample_entropy_bpiint = np.corrcoef(x,y)
print(corr_sample_entropy_bpiint)
bf_sample_entropy_bpiint = pingouin.bayesfactor_pearson(corr_sample_entropy_bpiint[0][1], len(x))
bpiint_bf.append(bf_sample_entropy_bpiint)
print(bf_sample_entropy_bpiint)

print('------')
print('acf')
acf_bpiint = zip(acf_s1, bpi_mean_interference)
x = []; y=[]
for point in list(acf_bpiint):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_acf_bpiint = np.corrcoef(x,y)
print(corr_acf_bpiint)
bf_acf_bpiint = pingouin.bayesfactor_pearson(corr_acf_bpiint[0][1], len(x))
bpiint_bf.append(bf_acf_bpiint)
print(bf_acf_bpiint)

print('------')
print('smape')
smape_bpiint = zip(smape_s1, bpi_mean_interference)
x = []; y=[]
for point in list(smape_bpiint):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_smape_bpiint = np.corrcoef(x,y)
print(corr_smape_bpiint)
bf_smape_bpiint = pingouin.bayesfactor_pearson(corr_smape_bpiint[0][1], len(x))
bpiint_bf.append(bf_smape_bpiint)
print(bf_smape_bpiint)

In [ ]:
back_pain = df_s1_normal['Back Pain'].values == 'Yes'
print('mean')
start_total = df_s1_normal['START total score'].iloc[back_pain].values
mean_start = zip(mean_s1[back_pain], start_total)
x = []; y=[]
for point in list(mean_start):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_mean_start = np.corrcoef(x,y)
print(corr_mean_start)
start_bf = []
bf_mean_start = pingouin.bayesfactor_pearson(corr_mean_start[0][1], len(x))
start_bf.append(bf_mean_start)
print(bf_mean_start)
print('-----')
print('variance')
var_start = zip(var_s1[back_pain], start_total)
x = []; y=[]
for point in list(var_start):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_var_start = np.corrcoef(x,y)
print(corr_var_start)
bf_var_start = pingouin.bayesfactor_pearson(corr_var_start[0][1], len(x))
start_bf.append(bf_var_start)
print(bf_var_start)

print('-----')
print('CV')
fano_start = zip(cv_s1[back_pain], start_total)
x = []; y=[]
for point in list(fano_start):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_fano_start = np.corrcoef(x,y)
print(corr_fano_start)
bf_fano_start = pingouin.bayesfactor_pearson(corr_fano_start[0][1], len(x))
start_bf.append(bf_fano_start)
print(bf_fano_start)

print('-----')
print('PE')
entropy_start = zip(entropy_s1[back_pain], start_total)
x = []; y=[]
for point in list(entropy_start):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_entropy_start = np.corrcoef(x,y)
print(corr_entropy_start)
bf_entropy_start = pingouin.bayesfactor_pearson(corr_entropy_start[0][1], len(x))
start_bf.append(bf_entropy_start)
print(bf_entropy_start)

print('-----')
print('SampEn')
sample_entropy_start = zip(sample_entropy_s1[back_pain], start_total)
x = []; y=[]
for point in list(sample_entropy_start):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_sample_entropy_start = np.corrcoef(x,y)
print(corr_sample_entropy_start)
bf_sample_entropy_start = pingouin.bayesfactor_pearson(corr_sample_entropy_start[0][1], len(x))
start_bf.append(bf_sample_entropy_start)
print(bf_sample_entropy_start)

print('-----')
print('acf')
acf_start = zip(acf_s1[back_pain], start_total)
x = []; y=[]
for point in list(acf_start):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_acf_start = np.corrcoef(x,y)
print(corr_acf_start)
bf_acf_start = pingouin.bayesfactor_pearson(corr_acf_start[0][1], len(x))
start_bf.append(bf_acf_start)
print(bf_acf_start)

print('-----')
print('smape')
smape_start = zip(smape_s1[back_pain], start_total)
x = []; y=[]
for point in list(smape_start):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_smape_start = np.corrcoef(x,y)
print(corr_smape_start)
bf_smape_start = pingouin.bayesfactor_pearson(corr_smape_start[0][1], len(x))
start_bf.append(bf_smape_start)
print(bf_smape_start)



In [ ]:
start_sub = df_s1_normal['START sub score'].iloc[back_pain].values
print('mean')
mean_startsub = zip(mean_s1, start_sub)
x = []; y=[]
for point in list(mean_startsub):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_mean_startsub = np.corrcoef(x,y)
print(corr_mean_startsub)
startsub_bf = []
bf_mean_startsub = pingouin.bayesfactor_pearson(corr_mean_startsub[0][1], len(x))
startsub_bf.append(bf_mean_startsub)
print(bf_mean_startsub)
print('-----')
print('variance')
var_startsub = zip(var_s1, start_sub)
x = []; y=[]
for point in list(var_startsub):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_var_startsub = np.corrcoef(x,y)
print(corr_var_startsub)
bf_var_startsub = pingouin.bayesfactor_pearson(corr_var_startsub[0][1], len(x))
startsub_bf.append(bf_var_startsub)
print(bf_var_startsub)

print('-----')
print('CV')
fano_startsub = zip(cv_s1, start_sub)
x = []; y=[]
for point in list(fano_startsub):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_fano_startsub = np.corrcoef(x,y)
print(corr_fano_startsub)
bf_fano_startsub = pingouin.bayesfactor_pearson(corr_fano_startsub[0][1], len(x))
startsub_bf.append(bf_fano_startsub)
print(bf_fano_startsub)

print('-----')
print('PE')
entropy_startsub = zip(entropy_s1, start_sub)
x = []; y=[]
for point in list(entropy_startsub):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_entropy_startsub = np.corrcoef(x,y)
print(corr_entropy_startsub)
bf_entropy_startsub = pingouin.bayesfactor_pearson(corr_entropy_startsub[0][1], len(x))
startsub_bf.append(bf_entropy_startsub)
print(bf_entropy_startsub)

print('-----')
print('SampEn')
sample_entropy_startsub = zip(sample_entropy_s1, start_sub)
x = []; y=[]
for point in list(sample_entropy_startsub):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_sample_entropy_startsub = np.corrcoef(x,y)
print(corr_sample_entropy_startsub)
bf_sample_entropy_startsub = pingouin.bayesfactor_pearson(corr_sample_entropy_startsub[0][1], len(x))
startsub_bf.append(bf_sample_entropy_startsub)
print(bf_sample_entropy_startsub)

print('-----')
print('acf')
acf_startsub = zip(acf_s1, start_sub)
x = []; y=[]
for point in list(acf_startsub):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_acf_startsub = np.corrcoef(x,y)
print(corr_acf_startsub)
bf_acf_startsub = pingouin.bayesfactor_pearson(corr_acf_startsub[0][1], len(x))
startsub_bf.append(bf_acf_startsub)
print(bf_acf_startsub)
print('-----')
print('smape')
smape_startsub = zip(smape_s1, start_sub)
x = []; y=[]
for point in list(smape_startsub):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_smape_startsub = np.corrcoef(x,y)
print(corr_smape_startsub)
bf_smape_startsub = pingouin.bayesfactor_pearson(corr_smape_startsub[0][1], len(x))
startsub_bf.append(bf_smape_startsub)
print(bf_smape_startsub)


## section 2 (pcs_total_bf, pcs_rum_bf, pcs_mag_bf, pcs_help_bf, gad7_bf, phq9_bf)

In [ ]:
index_s2_normal = []
for i in range(len(index_s2)):
    if len(index_s2[i]) == 1:
        index_s2_normal.append(index_s2[i][0])
df_s2_normal = df_questionnaire_s2.iloc[index_s2_normal]
participant_list_s2_questionnaire = df_s2_normal.PROLIFIC_PID.values#.to_list()
s2_questionnaire_index = []
for i in range(len(participant_list)):
    if participant_list[i] in participant_list_s2_questionnaire:
        s2_questionnaire_index.append(i)
mean_s2 = np.array(mean_overall)[s2_questionnaire_index]
var_s2 = np.array(var_combined_list_all)[s2_questionnaire_index]
entropy_s2 = np.array(entropy_overall)[s2_questionnaire_index]
cv_s2 = np.array(cv_combined_list)[s2_questionnaire_index]
acf_s2 = np.array(char_time_scale_mean)[s2_questionnaire_index]
smape_s2 = np.array(smape_mean_all)[s2_questionnaire_index]
sample_entropy_s2 = np.array(sample_entropy_overall)[s2_questionnaire_index]


In [ ]:
pcs_total = df_s2_normal['Total PCS Score'].values
print('mean')
mean_pcs_total = zip(mean_s2, pcs_total)
x = []; y=[]
for point in list(mean_pcs_total):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_mean_pcs_total = np.corrcoef(x,y)
print(corr_mean_pcs_total)
pcs_total_bf = []
bf_mean_pcs_total = pingouin.bayesfactor_pearson(corr_mean_pcs_total[0][1], len(x))
pcs_total_bf.append(bf_mean_pcs_total)
print(bf_mean_pcs_total)
print('-----')
print('variance')
var_pcs_total = zip(var_s2, pcs_total)
x = []; y=[]
for point in list(var_pcs_total):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_var_pcs_total = np.corrcoef(x,y)
print(corr_var_pcs_total)
bf_var_pcs_total = pingouin.bayesfactor_pearson(corr_var_pcs_total[0][1], len(x))
pcs_total_bf.append(bf_var_pcs_total)
print(bf_var_pcs_total)

print('-----')
print('CV')
fano_pcs_total = zip(cv_s2, pcs_total)
x = []; y=[]
for point in list(fano_pcs_total):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
a, b = np.polyfit(x, y, 1)
corr_fano_pcs_total = np.corrcoef(x,y)
print(corr_fano_pcs_total)
bf_fano_pcs_total = pingouin.bayesfactor_pearson(corr_fano_pcs_total[0][1], len(x))
pcs_total_bf.append(bf_fano_pcs_total)
print(bf_fano_pcs_total)

print('-----')
print('PE')
entropy_pcs_total = zip(entropy_s2, pcs_total)
x = []; y=[]
for point in list(entropy_pcs_total):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_entropy_pcs_total = np.corrcoef(x,y)
print(corr_entropy_pcs_total)
bf_entropy_pcs_total = pingouin.bayesfactor_pearson(corr_entropy_pcs_total[0][1], len(x))
pcs_total_bf.append(bf_entropy_pcs_total)
print(bf_entropy_pcs_total)

print('-----')
print('SampEn')
sample_entropy_pcs_total = zip(sample_entropy_s2, pcs_total)
x = []; y=[]
for point in list(sample_entropy_pcs_total):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_sample_entropy_pcs_total = np.corrcoef(x,y)
print(corr_sample_entropy_pcs_total)
bf_sample_entropy_pcs_total = pingouin.bayesfactor_pearson(corr_sample_entropy_pcs_total[0][1], len(x))
pcs_total_bf.append(bf_sample_entropy_pcs_total)
print(bf_sample_entropy_pcs_total)
print('-----')
print('acf')
acf_pcs_total = zip(acf_s2, pcs_total)
x = []; y=[]
for point in list(acf_pcs_total):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_acf_pcs_total = np.corrcoef(x,y)
print(corr_acf_pcs_total)
bf_acf_pcs_total = pingouin.bayesfactor_pearson(corr_acf_pcs_total[0][1], len(x))
pcs_total_bf.append(bf_acf_pcs_total)
print(bf_acf_pcs_total)
print('-----')
print('smape')
smape_pcs_total = zip(smape_s2, pcs_total)
x = []; y=[]
for point in list(smape_pcs_total):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_smape_pcs_total = np.corrcoef(x,y)
print(corr_smape_pcs_total)
bf_smape_pcs_total = pingouin.bayesfactor_pearson(corr_smape_pcs_total[0][1], len(x))
pcs_total_bf.append(bf_smape_pcs_total)
print(bf_smape_pcs_total)


In [ ]:
pcs_rum = df_s2_normal['PCS Rumination Score'].values
print('mean')
mean_pcs_rum = zip(mean_s2, pcs_rum)
x = []; y=[]
for point in list(mean_pcs_rum):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_mean_pcs_rum = np.corrcoef(x,y)
print(corr_mean_pcs_rum)
pcs_rum_bf = []
bf_mean_pcs_rum = pingouin.bayesfactor_pearson(corr_mean_pcs_rum[0][1], len(x))
pcs_rum_bf.append(bf_mean_pcs_rum)
print(bf_mean_pcs_rum)
print('-----')
print('variance')
var_pcs_rum = zip(var_s2, pcs_rum)
x = []; y=[]
for point in list(var_pcs_rum):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_var_pcs_rum = np.corrcoef(x,y)
print(corr_var_pcs_rum)
bf_var_pcs_rum = pingouin.bayesfactor_pearson(corr_var_pcs_rum[0][1], len(x))
pcs_rum_bf.append(bf_var_pcs_rum)
print(bf_var_pcs_rum)

print('-----')
print('CV')
fano_pcs_rum = zip(cv_s2, pcs_rum)
x = []; y=[]
for point in list(fano_pcs_rum):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_fano_pcs_rum = np.corrcoef(x,y)
print(corr_fano_pcs_rum)
bf_fano_pcs_rum = pingouin.bayesfactor_pearson(corr_fano_pcs_rum[0][1], len(x))
pcs_rum_bf.append(bf_fano_pcs_rum)
print(bf_fano_pcs_rum)

print('-----')
print('PE')
entropy_pcs_rum = zip(entropy_s2, pcs_rum)
x = []; y=[]
for point in list(entropy_pcs_rum):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_entropy_pcs_rum = np.corrcoef(x,y)
print(corr_entropy_pcs_rum)
bf_entropy_pcs_rum = pingouin.bayesfactor_pearson(corr_entropy_pcs_rum[0][1], len(x))
pcs_rum_bf.append(bf_entropy_pcs_rum)
print(bf_entropy_pcs_rum)

print('-----')
print('SampEn')
sample_entropy_pcs_rum = zip(sample_entropy_s2, pcs_rum)
x = []; y=[]
for point in list(sample_entropy_pcs_rum):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_sample_entropy_pcs_rum = np.corrcoef(x,y)
print(corr_sample_entropy_pcs_rum)
bf_sample_entropy_pcs_rum = pingouin.bayesfactor_pearson(corr_sample_entropy_pcs_rum[0][1], len(x))
pcs_rum_bf.append(bf_sample_entropy_pcs_rum)
print(bf_sample_entropy_pcs_rum)

print('-----')
print('acf')
acf_pcs_rum = zip(acf_s2, pcs_rum)
x = []; y=[]
for point in list(acf_pcs_rum):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_acf_pcs_rum = np.corrcoef(x,y)
print(corr_acf_pcs_rum)
bf_acf_pcs_rum = pingouin.bayesfactor_pearson(corr_acf_pcs_rum[0][1], len(x))
pcs_rum_bf.append(bf_acf_pcs_rum)
print(bf_acf_pcs_rum)

print('-----')
print('smape')
smape_pcs_rum = zip(smape_s2, pcs_rum)
x = []; y=[]
for point in list(smape_pcs_rum):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_smape_pcs_rum = np.corrcoef(x,y)
print(corr_smape_pcs_rum)
bf_smape_pcs_rum = pingouin.bayesfactor_pearson(corr_smape_pcs_rum[0][1], len(x))
pcs_rum_bf.append(bf_smape_pcs_rum)
print(bf_smape_pcs_rum)



In [ ]:
pcs_mag = df_s2_normal['PCS Magnification Score'].values
print('mean')
mean_pcs_mag = zip(mean_s2, pcs_mag)
x = []; y=[]
for point in list(mean_pcs_mag):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_mean_pcs_mag = np.corrcoef(x,y)
print(corr_mean_pcs_mag)
pcs_mag_bf = []
bf_mean_pcs_mag = pingouin.bayesfactor_pearson(corr_mean_pcs_mag[0][1], len(x))
pcs_mag_bf.append(bf_mean_pcs_mag)
print(bf_mean_pcs_mag)
print('-----')
print('variance')
var_pcs_mag = zip(var_s2, pcs_mag)
x = []; y=[]
for point in list(var_pcs_mag):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_var_pcs_mag = np.corrcoef(x,y)
print(corr_var_pcs_mag)
bf_var_pcs_mag = pingouin.bayesfactor_pearson(corr_var_pcs_mag[0][1], len(x))
pcs_mag_bf.append(bf_var_pcs_mag)
print(bf_var_pcs_mag)

print('-----')
print('CV')
fano_pcs_mag = zip(cv_s2, pcs_mag)
x = []; y=[]
for point in list(fano_pcs_mag):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_fano_pcs_mag = np.corrcoef(x,y)
print(corr_fano_pcs_mag)
bf_fano_pcs_mag = pingouin.bayesfactor_pearson(corr_fano_pcs_mag[0][1], len(x))
pcs_mag_bf.append(bf_fano_pcs_mag)
print(bf_fano_pcs_mag)

print('-----')
print('PE')
entropy_pcs_mag = zip(entropy_s2, pcs_mag)
x = []; y=[]
for point in list(entropy_pcs_mag):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_entropy_pcs_mag = np.corrcoef(x,y)
print(corr_entropy_pcs_mag)
bf_entropy_pcs_mag = pingouin.bayesfactor_pearson(corr_entropy_pcs_mag[0][1], len(x))
pcs_mag_bf.append(bf_entropy_pcs_mag)
print(bf_entropy_pcs_mag)

print('-----')
print('SampEn')
sample_entropy_pcs_mag = zip(sample_entropy_s2, pcs_mag)
x = []; y=[]
for point in list(sample_entropy_pcs_mag):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_sample_entropy_pcs_mag = np.corrcoef(x,y)
print(corr_sample_entropy_pcs_mag)
bf_sample_entropy_pcs_mag = pingouin.bayesfactor_pearson(corr_sample_entropy_pcs_mag[0][1], len(x))
pcs_mag_bf.append(bf_sample_entropy_pcs_mag)
print(bf_sample_entropy_pcs_mag)

print('-----')
print('acf')
acf_pcs_mag = zip(acf_s2, pcs_mag)
x = []; y=[]
for point in list(acf_pcs_mag):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_acf_pcs_mag = np.corrcoef(x,y)
print(corr_acf_pcs_mag)
bf_acf_pcs_mag = pingouin.bayesfactor_pearson(corr_acf_pcs_mag[0][1], len(x))
pcs_mag_bf.append(bf_acf_pcs_mag)
print(bf_acf_pcs_mag)
print('-----')
print('smape')
smape_pcs_mag = zip(smape_s2, pcs_mag)
x = []; y=[]
for point in list(smape_pcs_mag):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_smape_pcs_mag = np.corrcoef(x,y)
print(corr_smape_pcs_mag)
bf_smape_pcs_mag = pingouin.bayesfactor_pearson(corr_smape_pcs_mag[0][1], len(x))
pcs_mag_bf.append(bf_smape_pcs_mag)
print(bf_smape_pcs_mag)


In [ ]:
pcs_help = df_s2_normal['PCS Helplessness Score'].values
print('mean')
mean_pcs_help = zip(mean_s2, pcs_help)
x = []; y=[]
for point in list(mean_pcs_help):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_mean_pcs_help = np.corrcoef(x,y)
print(corr_mean_pcs_help)
pcs_help_bf = []
bf_mean_pcs_help = pingouin.bayesfactor_pearson(corr_mean_pcs_help[0][1], len(x))
pcs_help_bf.append(bf_mean_pcs_help)
print(bf_mean_pcs_help)
print('-----')
print('variance')
var_pcs_help = zip(var_s2, pcs_help)
x = []; y=[]
for point in list(var_pcs_help):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_var_pcs_help = np.corrcoef(x,y)
print(corr_var_pcs_help)
bf_var_pcs_help = pingouin.bayesfactor_pearson(corr_var_pcs_help[0][1], len(x))
pcs_help_bf.append(bf_var_pcs_help)
print(bf_var_pcs_help)

print('-----')
print('CV')
fano_pcs_help = zip(cv_s2, pcs_help)
x = []; y=[]
for point in list(fano_pcs_help):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_fano_pcs_help = np.corrcoef(x,y)
print(corr_fano_pcs_help)
bf_fano_pcs_help = pingouin.bayesfactor_pearson(corr_fano_pcs_help[0][1], len(x))
pcs_help_bf.append(bf_fano_pcs_help)
print(bf_fano_pcs_help)

print('-----')
print('PE')
entropy_pcs_help = zip(entropy_s2, pcs_help)
x = []; y=[]
for point in list(entropy_pcs_help):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_entropy_pcs_help = np.corrcoef(x,y)
print(corr_entropy_pcs_help)
bf_entropy_pcs_help = pingouin.bayesfactor_pearson(corr_entropy_pcs_help[0][1], len(x))
pcs_help_bf.append(bf_entropy_pcs_help)
print(bf_entropy_pcs_help)

print('-----')
print('SampEn')
sample_entropy_pcs_help = zip(sample_entropy_s2, pcs_help)
x = []; y=[]
for point in list(sample_entropy_pcs_help):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_sample_entropy_pcs_help = np.corrcoef(x,y)
print(corr_sample_entropy_pcs_help)
bf_sample_entropy_pcs_help = pingouin.bayesfactor_pearson(corr_sample_entropy_pcs_help[0][1], len(x))
pcs_help_bf.append(bf_sample_entropy_pcs_help)
print(bf_sample_entropy_pcs_help)

print('-----')
print('acf')
acf_pcs_help = zip(acf_s2, pcs_help)
x = []; y=[]
for point in list(acf_pcs_help):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_acf_pcs_help = np.corrcoef(x,y)
print(corr_acf_pcs_help)
bf_acf_pcs_help = pingouin.bayesfactor_pearson(corr_acf_pcs_help[0][1], len(x))
pcs_help_bf.append(bf_acf_pcs_help)
print(bf_acf_pcs_help)
print('-----')
print('smape')
smape_pcs_help = zip(smape_s2, pcs_help)
x = []; y=[]
for point in list(smape_pcs_help):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_smape_pcs_help = np.corrcoef(x,y)
print(corr_smape_pcs_help)
bf_smape_pcs_help = pingouin.bayesfactor_pearson(corr_smape_pcs_help[0][1], len(x))
pcs_help_bf.append(bf_smape_pcs_help)
print(bf_smape_pcs_help)


In [ ]:
gad7_total = df_s2_normal['Total GAD7 Score'].values
print('mean')
mean_gad7 = zip(mean_s2, gad7_total)
x = []; y=[]
for point in list(mean_gad7):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_mean_gad7 = np.corrcoef(x,y)
print(corr_mean_gad7)
a, b = np.polyfit(x, y, 1)
gad7_bf = []
bf_mean_gad7 = pingouin.bayesfactor_pearson(corr_mean_gad7[0][1], len(x))
gad7_bf.append(bf_mean_gad7)
print(bf_mean_gad7)
print('-----')
print('variance')
var_gad7 = zip(var_s2, gad7_total)
x = []; y=[]
for point in list(var_gad7):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_var_gad7 = np.corrcoef(x,y)
print(corr_var_gad7)
bf_var_gad7 = pingouin.bayesfactor_pearson(corr_var_gad7[0][1], len(x))
gad7_bf.append(bf_var_gad7)
print(bf_var_gad7)

print('-----')
print('CV')
fano_gad7 = zip(cv_s2, gad7_total)
x = []; y=[]
for point in list(fano_gad7):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_fano_gad7 = np.corrcoef(x,y)
print(corr_fano_gad7)
bf_fano_gad7 = pingouin.bayesfactor_pearson(corr_fano_gad7[0][1], len(x))
gad7_bf.append(bf_fano_gad7)
print(bf_fano_gad7)

print('-----')
print('PE')
entropy_gad7 = zip(entropy_s2, gad7_total)
x = []; y=[]
for point in list(entropy_gad7):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_entropy_gad7 = np.corrcoef(x,y)
print(corr_entropy_gad7)
bf_entropy_gad7 = pingouin.bayesfactor_pearson(corr_entropy_gad7[0][1], len(x))
gad7_bf.append(bf_entropy_gad7)
print(bf_entropy_gad7)

print('-----')
print('SampEn')
sample_entropy_gad7 = zip(sample_entropy_s2, gad7_total)
x = []; y=[]
for point in list(sample_entropy_gad7):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_sample_entropy_gad7 = np.corrcoef(x,y)
print(corr_sample_entropy_gad7)
bf_sample_entropy_gad7 = pingouin.bayesfactor_pearson(corr_sample_entropy_gad7[0][1], len(x))
gad7_bf.append(bf_sample_entropy_gad7)
print(bf_sample_entropy_gad7)

print('-----')
print('acf')
acf_gad7 = zip(acf_s2, gad7_total)
x = []; y=[]
for point in list(acf_gad7):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_acf_gad7 = np.corrcoef(x,y)
print(corr_acf_gad7)
bf_acf_gad7 = pingouin.bayesfactor_pearson(corr_acf_gad7[0][1], len(x))
gad7_bf.append(bf_acf_gad7)
print(bf_acf_gad7)
print('-----')
print('smape')
smape_gad7 = zip(smape_s2, gad7_total)
x = []; y=[]
for point in list(smape_gad7):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_smape_gad7 = np.corrcoef(x,y)
print(corr_smape_gad7)
bf_smape_gad7 = pingouin.bayesfactor_pearson(corr_smape_gad7[0][1], len(x))
gad7_bf.append(bf_smape_gad7)
print(bf_smape_gad7)


In [ ]:
phq9_total = df_s2_normal['Total PHQ9 Score'].values
print('mean')
mean_phq9 = zip(mean_s2, phq9_total)
x = []; y=[]
for point in list(mean_phq9):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_mean_phq9 = np.corrcoef(x,y)
print(corr_mean_phq9)
a, b = np.polyfit(x, y, 1)
phq9_bf = []
bf_mean_phq9 = pingouin.bayesfactor_pearson(corr_mean_phq9[0][1], len(x))
phq9_bf.append(bf_mean_phq9)
print(bf_mean_phq9)
print('-----')
print('variance')
var_phq9 = zip(var_s2, phq9_total)
x = []; y=[]
for point in list(var_phq9):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_var_phq9 = np.corrcoef(x,y)
print(corr_var_phq9)
a, b = np.polyfit(x, y, 1)
bf_var_phq9 = pingouin.bayesfactor_pearson(corr_var_phq9[0][1], len(x))
phq9_bf.append(bf_var_phq9)
print(bf_var_phq9)

print('-----')
print('CV')
fano_phq9 = zip(cv_s2, phq9_total)
x = []; y=[]
for point in list(fano_phq9):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_fano_phq9 = np.corrcoef(x,y)
bf_fano_phq9 = pingouin.bayesfactor_pearson(corr_fano_phq9[0][1], len(x))
phq9_bf.append(bf_fano_phq9)
print(corr_fano_phq9)
print(bf_fano_phq9)


print('-----')
print('PE')
entropy_phq9 = zip(entropy_s2, phq9_total)
x = []; y=[]
for point in list(entropy_phq9):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_entropy_phq9 = np.corrcoef(x,y)
print(corr_entropy_phq9)
bf_entropy_phq9 = pingouin.bayesfactor_pearson(corr_entropy_phq9[0][1], len(x))
phq9_bf.append(bf_entropy_phq9)
print(bf_entropy_phq9)

print('-----')
print('SampEn')
sample_entropy_phq9 = zip(sample_entropy_s2, phq9_total)
x = []; y=[]
for point in list(sample_entropy_phq9):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_sample_entropy_phq9 = np.corrcoef(x,y)
print(corr_sample_entropy_phq9)
bf_sample_entropy_phq9 = pingouin.bayesfactor_pearson(corr_sample_entropy_phq9[0][1], len(x))
phq9_bf.append(bf_sample_entropy_phq9)
print(bf_sample_entropy_phq9)

print('-----')
print('acf')
acf_phq9 = zip(acf_s2, phq9_total)
x = []; y=[]
for point in list(acf_phq9):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_acf_phq9 = np.corrcoef(x,y)
print(corr_acf_phq9)
bf_acf_phq9 = pingouin.bayesfactor_pearson(corr_acf_phq9[0][1], len(x))
phq9_bf.append(bf_acf_phq9)
print(bf_acf_phq9)

print('-----')
print('smape')
smape_phq9 = zip(smape_s2, phq9_total)
x = []; y=[]
for point in list(smape_phq9):
   x.append(point[0])
   y.append(point[1])
for i in range(len(y)):
   if pd.isnull(y[i]):
      x[i] = np.NaN
   if pd.isnull(x[i]):
      y[i] = np.NaN
x = pd.Series(x).dropna().to_list()
y = pd.Series(y).dropna().to_list()
corr_smape_phq9 = np.corrcoef(x,y)
print(corr_smape_phq9)
bf_smape_phq9 = pingouin.bayesfactor_pearson(corr_smape_phq9[0][1], len(x))
phq9_bf.append(bf_smape_phq9)
print(bf_smape_phq9)

# Heat map code

In [ ]:
plt.rcParams['figure.figsize'] = (14,14)
plt.rcParams['font.size'] = '14'
total_bf = np.vstack([msk_bf, msk_lastq_bf, bpisev_bf, bpiint_bf, start_bf, startsub_bf, pcs_total_bf, pcs_rum_bf, pcs_mag_bf, pcs_help_bf, gad7_bf, phq9_bf])
sns.heatmap(total_bf, norm=LogNorm(), xticklabels=['Mean', 'Variance', 'CV','PE', 'SampEn','ACF time scale', 'SMAPE', 'Fractal dimension'], cmap="YlGnBu", annot = True,
            yticklabels = ['MSK score', 'Number of Physically Active Days','BPI mean severity', 'BPI mean interference', 'total STarT', 'STarT sub Score', 'Total PCS score', 'PCS rumination score', 'PCS magnification score', 'PCS helplessness score', 'Total GAD7', 'Total PHQ9'])
plt.title('Heat map with BF10 values of questionnaires & various features')
plt.tight_layout()

## Test-retest reliability

In [ ]:
data_d1 = pd.read_pickle('../Data/descriptive_data/DescriptiveData.pkl')
data_d2 = pd.read_pickle('../Data/data_file/D2_puregooddata_split_31032023.pkl')
participant_list_d2 = data_d2['PID'].unique()
data_d1_in_d2 = data_d1[data_d1['PID'].isin(participant_list_d2)]
mean_overall_d1 = data_d1_in_d2['mean overall'].values
var_overall_d1 = data_d1_in_d2['variance overall'].values
cv_overall_d1 = data_d1_in_d2['CV overall'].values
pe_overall_d1 = data_d1_in_d2['permutation entropy overall'].values
sampen_overall_d1 = data_d1_in_d2['sample entropy overall'].values
fractal_overall_d1 = data_d1_in_d2['fractal dimension overall'].values
mean_all_d1 = data_d1_in_d2['mean'].values
var_all_d1 = data_d1_in_d2['variance'].values
cv_all_d1 = data_d1_in_d2['CV'].values
pe_all_d1 = data_d1_in_d2['permutation entropy'].values
sampen_all_d1 = data_d1_in_d2['sample entropy'].values
fractal_all_d1 = data_d1_in_d2['fractal dimension'].values
data_d2 = pd.read_pickle('../Data/descriptive_data/DescriptiveDataD2.pkl')
mean_overall_d2 = data_d2['mean overall'].values
var_overall_d2 = data_d2['variance overall'].values
cv_overall_d2 = data_d2['CV overall'].values
pe_overall_d2 = data_d2['permutation entropy overall'].values
sampen_overall_d2 = data_d2['sample entropy overall'].values
fractal_overall_d2 = data_d2['fractal dimension overall'].values
mean_all_d2 = data_d2['mean'].values
var_all_d2 = data_d2['variance'].values
cv_all_d2 = data_d2['CV'].values
pe_all_d2 = data_d2['permutation entropy'].values
sampen_all_d2 = data_d2['sample entropy'].values
fractal_all_d2 = data_d2['fractal dimension'].values

# Pearson coefficient method

In [ ]:
print('Pearson\'s Coefficient')
print('-----')
print('mean')
print(np.corrcoef(mean_overall_d1, mean_overall_d2))
print('-----')
print('variance')
print(np.corrcoef(var_overall_d1, var_overall_d2))
print('-----')
print('CV')
print(np.corrcoef(cv_overall_d1, cv_overall_d2))
print('-----')
print('PE')
print(np.corrcoef(pe_overall_d1, pe_overall_d2))
print('-----')
print('SampEn')
print(np.corrcoef(sampen_overall_d1, sampen_overall_d2))
print('-----')
print('fractal')
print(np.corrcoef(fractal_overall_d1, fractal_overall_d2))

## ICC method
### Mean

In [ ]:
session_1_mean_df = pd.DataFrame({'mean': mean_overall_d1, 'participant': range(len(mean_overall_d1)), 'session': np.zeros(len(mean_overall_d1))})
session_2_mean_df = pd.DataFrame({'mean': mean_overall_d2, 'participant': range(len(mean_overall_d2)), 'session': np.ones(len(mean_overall_d2))})
mean_df = pd.concat([session_1_mean_df,session_2_mean_df])
icc_mean_overall = pg.intraclass_corr(data=mean_df, targets = 'participant', raters = 'session', ratings='mean')
icc_mean_overall.set_index('Type')
mean_all_sep_d1 = []
mean_all_participant_d1 = []
for i in range(len(mean_all_d1)):
    for j in range(len(mean_all_d1[i])):
        mean_all_sep_d1.append(mean_all_d1[i][j])
        mean_all_participant_d1.append(i)
mean_all_sep_d2 = []
mean_all_participant_d2 = []
for i in range(len(mean_all_d2)):
    for j in range(len(mean_all_d2[i])):
        mean_all_sep_d2.append(mean_all_d2[i][j])
        mean_all_participant_d2.append(i)
session_1_mean_df_all = pd.DataFrame({'mean': mean_all_sep_d1, 'participant': mean_all_participant_d1, 'session': np.zeros(len(mean_all_sep_d1))})
session_2_mean_df_all = pd.DataFrame({'mean': mean_all_sep_d2, 'participant': mean_all_participant_d2, 'session': np.ones(len(mean_all_sep_d2))})
mean_df_all = pd.concat([session_1_mean_df_all,session_2_mean_df_all])
icc_mean_all = pg.intraclass_corr(data=mean_df_all, targets = 'participant', raters = 'session', ratings='mean')
icc_mean_all.set_index('Type')

### Variance

In [ ]:
session_1_var_df = pd.DataFrame({'var': var_overall_d1, 'participant': range(len(var_overall_d1)), 'session': np.zeros(len(var_overall_d1))})
session_2_var_df = pd.DataFrame({'var': var_overall_d2, 'participant': range(len(var_overall_d2)), 'session': np.ones(len(var_overall_d2))})
var_df = pd.concat([session_1_var_df,session_2_var_df])
icc_var_overall = pg.intraclass_corr(data=var_df, targets = 'participant', raters = 'session', ratings='var')
icc_var_overall.set_index('Type')
var_all_sep_d1 = []
var_all_participant_d1 = []
for i in range(len(var_all_d1)):
    for j in range(len(var_all_d1[i])):
        var_all_sep_d1.append(var_all_d1[i][j])
        var_all_participant_d1.append(i)
var_all_sep_d2 = []
var_all_participant_d2 = []
for i in range(len(var_all_d2)):
    for j in range(len(var_all_d2[i])):
        var_all_sep_d2.append(var_all_d2[i][j])
        var_all_participant_d2.append(i)
session_1_var_df_all = pd.DataFrame({'var': var_all_sep_d1, 'participant': var_all_participant_d1, 'session': np.zeros(len(var_all_sep_d1))})
session_2_var_df_all = pd.DataFrame({'var': var_all_sep_d2, 'participant': var_all_participant_d2, 'session': np.ones(len(var_all_sep_d2))})
var_df_all = pd.concat([session_1_var_df_all,session_2_var_df_all])
icc_var_all = pg.intraclass_corr(data=var_df_all, targets = 'participant', raters = 'session', ratings='var')
icc_var_all.set_index('Type')

### CV

In [ ]:
session_1_cv_df = pd.DataFrame({'cv': cv_overall_d1, 'participant': range(len(cv_overall_d1)), 'session': np.zeros(len(cv_overall_d1))})
session_2_cv_df = pd.DataFrame({'cv': cv_overall_d2, 'participant': range(len(cv_overall_d2)), 'session': np.ones(len(cv_overall_d2))})
cv_df = pd.concat([session_1_cv_df,session_2_cv_df])
icc_cv_overall = pg.intraclass_corr(data=cv_df, targets = 'participant', raters = 'session', ratings='cv')
icc_cv_overall.set_index('Type')
cv_all_sep_d1 = []
cv_all_participant_d1 = []
for i in range(len(cv_all_d1)):
    for j in range(len(cv_all_d1[i])):
        cv_all_sep_d1.append(cv_all_d1[i][j])
        cv_all_participant_d1.append(i)
cv_all_sep_d2 = []
cv_all_participant_d2 = []
for i in range(len(cv_all_d2)):
    for j in range(len(cv_all_d2[i])):
        cv_all_sep_d2.append(cv_all_d2[i][j])
        cv_all_participant_d2.append(i)
session_1_cv_df_all = pd.DataFrame({'cv': cv_all_sep_d1, 'participant': cv_all_participant_d1, 'session': np.zeros(len(cv_all_sep_d1))})
session_2_cv_df_all = pd.DataFrame({'cv': cv_all_sep_d2, 'participant': cv_all_participant_d2, 'session': np.ones(len(cv_all_sep_d2))})
cv_df_all = pd.concat([session_1_cv_df_all,session_2_cv_df_all])
icc_cv_all = pg.intraclass_corr(data=cv_df_all, targets = 'participant', raters = 'session', ratings='cv')
icc_cv_all.set_index('Type')

### PE

In [ ]:
session_1_pe_df = pd.DataFrame({'pe': pe_overall_d1, 'participant': range(len(pe_overall_d1)), 'session': np.zeros(len(pe_overall_d1))})
session_2_pe_df = pd.DataFrame({'pe': pe_overall_d2, 'participant': range(len(pe_overall_d2)), 'session': np.ones(len(pe_overall_d2))})
pe_df = pd.concat([session_1_pe_df,session_2_pe_df])
icc_pe_overall = pg.intraclass_corr(data=pe_df, targets = 'participant', raters = 'session', ratings='pe')
icc_pe_overall.set_index('Type')
pe_all_sep_d1 = []
pe_all_participant_d1 = []
for i in range(len(pe_all_d1)):
    for j in range(len(pe_all_d1[i])):
        pe_all_sep_d1.append(pe_all_d1[i][j])
        pe_all_participant_d1.append(i)
pe_all_sep_d2 = []
pe_all_participant_d2 = []
for i in range(len(pe_all_d2)):
    for j in range(len(pe_all_d2[i])):
        pe_all_sep_d2.append(pe_all_d2[i][j])
        pe_all_participant_d2.append(i)
session_1_pe_df_all = pd.DataFrame({'pe': pe_all_sep_d1, 'participant': pe_all_participant_d1, 'session': np.zeros(len(pe_all_sep_d1))})
session_2_pe_df_all = pd.DataFrame({'pe': pe_all_sep_d2, 'participant': pe_all_participant_d2, 'session': np.ones(len(pe_all_sep_d2))})
pe_df_all = pd.concat([session_1_pe_df_all,session_2_pe_df_all])
icc_pe_all = pg.intraclass_corr(data=pe_df_all, targets = 'participant', raters = 'session', ratings='pe')
icc_pe_all.set_index('Type')

### SampEn

In [ ]:
session_1_sampen_df = pd.DataFrame({'sampen': sampen_overall_d1, 'participant': range(len(sampen_overall_d1)), 'session': np.zeros(len(sampen_overall_d1))})
session_2_sampen_df = pd.DataFrame({'sampen': sampen_overall_d2, 'participant': range(len(sampen_overall_d2)), 'session': np.ones(len(sampen_overall_d2))})
sampen_df = pd.concat([session_1_sampen_df,session_2_sampen_df])
icc_sampen_overall = pg.intraclass_corr(data=sampen_df, targets = 'participant', raters = 'session', ratings='sampen')
icc_sampen_overall.set_index('Type')
sampen_all_sep_d1 = []
sampen_all_participant_d1 = []
for i in range(len(sampen_all_d1)):
    for j in range(len(sampen_all_d1[i])):
        sampen_all_sep_d1.append(sampen_all_d1[i][j])
        sampen_all_participant_d1.append(i)
sampen_all_sep_d2 = []
sampen_all_participant_d2 = []
for i in range(len(sampen_all_d2)):
    for j in range(len(sampen_all_d2[i])):
        sampen_all_sep_d2.append(sampen_all_d2[i][j])
        sampen_all_participant_d2.append(i)
session_1_sampen_df_all = pd.DataFrame({'sampen': sampen_all_sep_d1, 'participant': sampen_all_participant_d1, 'session': np.zeros(len(sampen_all_sep_d1))})
session_2_sampen_df_all = pd.DataFrame({'sampen': sampen_all_sep_d2, 'participant': sampen_all_participant_d2, 'session': np.ones(len(sampen_all_sep_d2))})
sampen_df_all = pd.concat([session_1_sampen_df_all,session_2_sampen_df_all])
icc_sampen_all = pg.intraclass_corr(data=sampen_df_all, targets = 'participant', raters = 'session', ratings='sampen')
icc_sampen_all.set_index('Type')

### Fractal

In [ ]:
session_1_fractal_df = pd.DataFrame({'fractal': fractal_overall_d1, 'participant': range(len(fractal_overall_d1)), 'session': np.zeros(len(fractal_overall_d1))})
session_2_fractal_df = pd.DataFrame({'fractal': fractal_overall_d2, 'participant': range(len(fractal_overall_d2)), 'session': np.ones(len(fractal_overall_d2))})
fractal_df = pd.concat([session_1_fractal_df,session_2_fractal_df])
icc_fractal_overall = pg.intraclass_corr(data=fractal_df, targets = 'participant', raters = 'session', ratings='fractal')
icc_fractal_overall.set_index('Type')
fractal_all_sep_d1 = []
fractal_all_participant_d1 = []
for i in range(len(fractal_all_d1)):
    for j in range(len(fractal_all_d1[i])):
        fractal_all_sep_d1.append(fractal_all_d1[i][j])
        fractal_all_participant_d1.append(i)
fractal_all_sep_d2 = []
fractal_all_participant_d2 = []
for i in range(len(fractal_all_d2)):
    for j in range(len(fractal_all_d2[i])):
        fractal_all_sep_d2.append(fractal_all_d2[i][j])
        fractal_all_participant_d2.append(i)
session_1_fractal_df_all = pd.DataFrame({'fractal': fractal_all_sep_d1, 'participant': fractal_all_participant_d1, 'session': np.zeros(len(fractal_all_sep_d1))})
session_2_fractal_df_all = pd.DataFrame({'fractal': fractal_all_sep_d2, 'participant': fractal_all_participant_d2, 'session': np.ones(len(fractal_all_sep_d2))})
fractal_df_all = pd.concat([session_1_fractal_df_all,session_2_fractal_df_all])
icc_fractal_all = pg.intraclass_corr(data=fractal_df_all, targets = 'participant', raters = 'session', ratings='fractal')
icc_fractal_all.set_index('Type')